# Setup Your Analysis Environment

1) Choose a directory to house your project in: 
```
.../<project-directory>
```


2) Create and activate a fresh Python3 virtual environment there: 
```
$ cd .../<project-directory>
$ python -m virtualenv env #Package tested on Python 3.6.8
$ source env/bin/activate
```

3) Download the `cosmicfish` package from Git: 
```
$ git clone git@github.com:ndeporzio/cosmicfish.git
```

4) Install the `cosmicfish` package. Note that its dependencies will install automatically.
```
$ cd cosmicfish
$ pip install . 
```

5) Launch Jupyter and open `tutorial.ipynb` notebook using Jupyter browser
```
$ jupyter notebook
```

# Setup Fiducial Cosmology

Import the analysis package.

In [ ]:
import cosmicfish as cf 

Import relevant python packages... 

In [ ]:
#Import relevant packages
import os
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt

Other setup steps... 

In [ ]:
#Instruct pyplot to use seaborn 
sns.set()

#Set project, data, CLASS directories 
projectdir = cf.correct_path("~/Desktop/test/")
datastore = os.path.join(projectdir, "data")
classpath = os.path.join(projectdir, "class")

Specify the values of your fiducial cosmology and other physical constants.

In [ ]:
#Setup fiducial cosmology, note change to N_ncdm... 
fid = {"A_s" : 2.2321e-9, 
        "n_s" : 0.96659,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.11271,
        "tau_reio" : 0.059888,
        "h" : 0.70148,
        "N_ncdm" : 0, #Note the change! 
        "z_pk" : 0.7} #Not necessary, but helps neatness later on

A_s = fid['A_s']
n_s = fid['n_s']
omega_b = fid['omega_b']
omega_cdm = fid['omega_cdm']
tau_reio = fid['tau_reio']
h = fid['h']
z_pk = fid['z_pk']

c = 2.9979e8 #Units of m/s
H = 1000. * 100. * h #Units of m/s/Mpc
kp = 0.05 * h #Units of 1/Mpc
t_cmb = 2.726 #Units of K

Specify the parameter values you'd like to use to compute the forecast...

In [ ]:
#Setup redshift bins, fiducial relica masses to sample
z_table = np.arange(0.7, 2.1, 0.1)
m_chi = 1.0

# Convergence Analysis

Before running the forecast, we want to ensure our cosmological parameters are well converged about the points we are interested in using to calculate Fisher matrices. To do so, we can use the `relic_convergence_analysis` class of `cosmicfish`. 

We pass to `relic_convergence_analysis` some fiducial cosmology, and then it will vary the parameters of that fiducial cosmology to values specified by the user and compute the corresponding power spectrum derivatives. You can choose to pass specific values of the fiducial cosmology, or choose to vary relative to the fiducial cosmology. 

First, specify the points in parameter space you'd like to sample: 

In [ ]:
T_chi_table = np.arange(0.002, 0.072, 0.002) #We pass specific values of T_ncdm
nonT_relvary = np.arange(1.00, 1.11, 0.01) #We pass relative values for the other parameters

Then, we generate instances of the `relic_convergence_analysis` class. It will search for pre-existing datasets for the specified points in parameter space in the datastore directory - otherwise, it will generate the dataset and place it in datastore. 

In [ ]:
conv1 = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, m_chi, classpath, datastore)
conv2 = cf.relic_convergence_analysis(fid, 'omega_b', 'rel', nonT_relvary, z_table, 0, classpath, datastore);
conv3 = cf.relic_convergence_analysis(fid, 'omega_cdm', 'rel', nonT_relvary, z_table, 0, classpath, datastore);
conv4 = cf.relic_convergence_analysis(fid, 'tau_reio', 'rel', nonT_relvary, z_table, 0, classpath, datastore);
conv5 = cf.relic_convergence_analysis(fid, 'h', 'rel', nonT_relvary, z_table, 0, classpath, datastore);

At this point, all the useful information generated in the convergence analysis is accessed through the methods of the `relic_convergence_analysis` objects. We use those methods to plot the results of the analysis...

### Convergence Power Spectra

In [ ]:
conv1.plot_ps(z_index=3)
conv2.plot_ps(z_index=3)
conv3.plot_ps(z_index=3)
conv4.plot_ps(z_index=3)
conv5.plot_ps(z_index=3)

### Convergence Derivatives of Power Spectra

In [ ]:
conv1.plot_dps(z_index=3)
conv2.plot_dps(z_index=3)
conv3.plot_dps(z_index=3)
conv4.plot_dps(z_index=3)
conv5.plot_dps(z_index=3)

### Convergence Difference of Derivatives of Power Spectra

In [ ]:
conv1.plot_delta_dps(z_index=3)
conv2.plot_delta_dps(z_index=3)
conv3.plot_delta_dps(z_index=3)
conv4.plot_delta_dps(z_index=3)
conv5.plot_delta_dps(z_index=3)

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.
# Analysis - No Commentary

In [ ]:
#Import relevant packages
import os
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import cosmicfish as cf


#Instruct pyplot to use seaborn 
sns.set()

#Set project, data, CLASS directories 
projectdir = cf.correct_path("~/Desktop/test/")
datastore = cf.correct_path("/Volumes/SSD01/data/")
classpath = os.path.join(projectdir, "class")

#Setup fiducial cosmology, note change to N_ncdm... 
fid = {"A_s" : 2.2321e-9, 
        "n_s" : 0.96659,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.11271,
        "tau_reio" : 0.059888,
        "h" : 0.70148,
        "N_ncdm" : 0, #Note the change! 
        "z_pk" : 0.7, 
        "T_cmb" : 2.726} #Not necessary, but helps neatness later on

A_s = fid['A_s']
n_s = fid['n_s']
omega_b = fid['omega_b']
omega_cdm = fid['omega_cdm']
tau_reio = fid['tau_reio']
h = fid['h']
z_pk = fid['z_pk']
T_cmb = fid['T_cmb'] #Units of K

c = 2.9979e8 #Units of m/s
H = 1000. * 100. * h #Units of m/s/Mpc
kp = 0.05 * h #Units of 1/Mpc


#Setup redshift bins, fiducial relica masses, relic temps to sample
z_table = np.arange(1.0, 1.1, 0.1)
m_chi = 0.1
T_chi_table = np.arange(0.1, 2.1, 0.1) / T_cmb
nonT_relvary = np.arange(1.01, 1.11, 0.01)

In [ ]:
conv1 = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, m_chi, classpath, datastore)
conv2 = cf.relic_convergence_analysis(fid, 'omega_b', 'rel', nonT_relvary, z_table, None, classpath, datastore);
conv3 = cf.relic_convergence_analysis(fid, 'omega_cdm', 'rel', nonT_relvary, z_table, None, classpath, datastore);
conv4 = cf.relic_convergence_analysis(fid, 'tau_reio', 'rel', nonT_relvary, z_table, None, classpath, datastore);
conv5 = cf.relic_convergence_analysis(fid, 'h', 'rel', np.arange(1.005, 1.055, 0.005), z_table, None, classpath, datastore);

In [ ]:
conv1.plot_ps(z_index=0, xscale='linear', plotdata=True)
conv2.plot_ps(z_index=0, xscale='linear', plotdata=True)
conv3.plot_ps(z_index=0, xscale='linear', plotdata=True)
conv4.plot_ps(z_index=0, xscale='linear', plotdata=True)
conv5.plot_ps(z_index=0, xscale='linear', plotdata=True)

print('..................................................................................')
print('..................................................................................')
print('..................................................................................')

conv1.plot_dps(z_index=0, xscale='linear')
conv2.plot_dps(z_index=0, xscale='linear')
conv3.plot_dps(z_index=0, xscale='linear')
conv4.plot_dps(z_index=0, xscale='linear')
conv5.plot_dps(z_index=0, xscale='linear')

print('..................................................................................')
print('..................................................................................')
print('..................................................................................')

conv1.plot_delta_dps(z_index=0, xscale='linear')
conv2.plot_delta_dps(z_index=0, xscale='linear')
conv3.plot_delta_dps(z_index=0, xscale='linear')
conv4.plot_delta_dps(z_index=0, xscale='linear')
conv5.plot_delta_dps(z_index=0, xscale='linear')

In [ ]:
conv1a = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, 0.1, classpath, datastore)
conv1b = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, 0.5, classpath, datastore)
conv1c = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, 1.0, classpath, datastore)
conv1d = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, 1.5, classpath, datastore)
conv1e = cf.relic_convergence_analysis(fid, 'T_ncdm', 'abs', T_chi_table, z_table, 2.5, classpath, datastore)

In [ ]:
#conv1a.plot_ps(z_index=0, xscale='linear', plotdata=True)
conv1a.plot_dps(z_index=0, xscale='linear')
conv1b.plot_dps(z_index=0, xscale='linear')
conv1c.plot_dps(z_index=0, xscale='linear')
conv1d.plot_dps(z_index=0, xscale='linear')
conv1e.plot_dps(z_index=0, xscale='linear')
#conv1a.plot_delta_dps(z_index=0, xscale='linear')

In [ ]:
conv1e.plot_delta_dps(z_index=0, xscale='linear')